# ARDDoS Data analysis for TFTP, SNMP and SSDP test traffic

In [ ]:
# Generic
import os
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import math
import re
import ipaddress
import kaleido

# For showing progress bar of for loops
from progressbar import Bar, ETA, \
    AdaptiveETA, Percentage, \
    ProgressBar 
widgets = [Percentage(),
            ' ', Bar(),
            ' ', ETA(),
            ' ', AdaptiveETA()]
pbar = ProgressBar(widgets=widgets)

# Own functions
from pcapreader import PcapReader
pcapToDf = PcapReader.pcapToDf

# For converting string to ip address in dataframe
from cyberpandas import to_ipaddress 
# For pretty printing dataframes
from tabulate import tabulate 
# For plotting
import plotly.express as px

# Functions

In [ ]:
def refineLatexTableStringScientificPaper(latexTableString):
    # Hacks for multiindex dataframe
    multiColFirst = latexTableString.find("\multicolumn")
    latexTableString = latexTableString[:multiColFirst+16] + 'c' + latexTableString[multiColFirst+17:]
    startRemove = latexTableString.find('\multicolumn',multiColFirst+1)
    latexTableString = latexTableString[:startRemove] + latexTableString[startRemove+18:]
    # Convert Level, Amplification factor and packets per sec to multirow:
    latexTableString = latexTableString[:latexTableString.find('Level')] + '\\multirow{2}{*}{Level}' +  latexTableString[latexTableString.find('Level')+len('Level'):]
    latexTableString = latexTableString[:latexTableString.find('Amplification factor')] + '\\multirow{2}{*}{\parbox{2.4cm}{\centering Amplification \\\\ Factor}}' +  latexTableString[latexTableString.find('Amplification factor')+len('Amplification factor'):]
    latexTableString = latexTableString[:latexTableString.find('Packets Per Second')] + '\\multirow{2}{*}{\parbox{3.4cm}{\centering Attacker Packets \\\\ Per Second}}' +  latexTableString[latexTableString.find('Packets Per Second')+len('Packets Per Second'):]
    # Adding a horizontal line for multindex table
    ## Find first \\ after the last column (packets per second)
    columnHeaderEndIndex = latexTableString.find('Second')
    latexTableString = latexTableString[:latexTableString.find('\\\\',columnHeaderEndIndex)+len('\\\\')] + '\cline{2-5}' + latexTableString[latexTableString.find('\\\\',columnHeaderEndIndex)+len('\\\\'):]
    # Add "tab" to label

    # Find caption and label and put at the bottom
    captionIndex = latexTableString.find("\caption")
    labelIndex = latexTableString.find('\label')
    labelStart = latexTableString.find('{',labelIndex)
    # Put 'tab' in label for figure
    latexTableString = latexTableString[:labelStart+1] + 'tab:' + latexTableString[labelStart+1:]
    captionEndIndex = latexTableString.find("}",labelIndex+1)+1
    endTableIndex = latexTableString.find("\end{tabular}")
    endTableEndIndex = endTableIndex + len("\end{tabular}")
    latexTableString = latexTableString[:captionIndex] + latexTableString[captionEndIndex:endTableEndIndex] + latexTableString[captionIndex:captionEndIndex] + latexTableString[endTableEndIndex:]
    return latexTableString

In [ ]:
## Get All pcap filenames:
def getFiles(baseDir):
    victimFilenames = []
    attackerFilenames = []
    reflectorFilenames = []
    for root, dirs, files in os.walk(baseDir):
        for file in files:
            if file.endswith(".pcapng"):
                if "victim" in file:
                    victimFilenames.append(os.path.join(root, file))
                if "attacker" in file:
                    attackerFilenames.append(os.path.join(root, file))
                if "reflector" in file:
                    reflectorFilenames.append(os.path.join(root,file))
    return attackerFilenames,victimFilenames,reflectorFilenames

In [ ]:
## Get attacker bytes sent
def getAttackStats(attackerFilenames,Protocol,useCachedBytes=False,BytesOrPackets="bytes",Tool="Own"):
    print("Getting Attacker bytes for " + Protocol + " from pcap files...")
    picklePath = os.path.dirname(os.path.dirname(attackerFilenames[0])) + '/AttackerBytes_' + Protocol + '_' + BytesOrPackets + '_' + Tool + '.pkl'
    if not useCachedBytes:
        pbar = ProgressBar(widgets=widgets)
        attackerStats = []
        for attackFile in pbar(attackerFilenames):
            # Get attack level
            level = int(re.search(r'\d+',attackFile).group(0)) # Find first number in string = level
            attackerDf = pcapToDf(attackFile,True,Tool) 
            if Protocol == "TFTP" or Protocol == "SSDP" or Protocol == "SNMP":
            # By finding the TFTP packets and summing the byte lengths we get the total number of bytes send by the attacker
                attackerDf = attackerDf.loc[attackerDf['Protocol'].isin([Protocol])]
            if BytesOrPackets == "bytes":
                attackerSent = attackerDf["Length"].sum()
            else:
                attackerSent = len(attackerDf)
            pktPerSec = len(attackerDf)/15 # All attacks are 15 sec
            attackerStats.append({'Level':level,'Attacker Outbound':attackerSent,'Packets Per Second':pktPerSec})
        attackerStats = pd.DataFrame(attackerStats).sort_values('Level')
        # Save to pickle file for fast reloading
        attackerStats.to_pickle(picklePath)
    else:
        if os.path.exists(picklePath):
            attackerStats = pd.read_pickle(picklePath)
        else:
            raise Exception("Error no cached pickle file. Run the function with useCachedBytes=False to recalcuate the df, and create the pickle file '" + os.path.basename(picklePath) + "'" )
 
    return attackerStats

In [ ]:
## Get Victim bytes received
def getVictimStats(victimFilenames,Protocol,useCachedBytes=False,BytesOrPackets="bytes",Tool="Own"):
    print("Getting Victim bytes for " + Protocol + " from pcap files...")
    picklePath = os.path.dirname(os.path.dirname(victimFilenames[0])) + '/VictimBytes' + Protocol + '_' + BytesOrPackets + '_' + Tool + '.pkl'
    if not useCachedBytes:
        pbar = ProgressBar(widgets=widgets)
        victimStats = []
        for victimFile in pbar(victimFilenames):
            # Get attack level
            level = int(re.search(r'\d+',victimFile).group(0)) # Find first number in string = level
            victimDf = pcapToDf(victimFile,True,Tool)
            if Protocol == "TFTP":
                # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
                victimDf = victimDf.loc[(victimDf['UDP Destination Port'] == 50040)]
            elif Protocol == "SNMP":
                victimDf = victimDf.loc[victimDf['Protocol'].isin([Protocol])]
            elif Protocol == "SSDP":
                victimDf = victimDf.loc[(victimDf['Protocol'] == Protocol) & (victimDf['Destination'] != ipaddress.ip_address('239.255.255.250'))]
            if BytesOrPackets == "bytes":
                victimReceived = victimDf["Length"].sum()
            else:
                victimReceived = len(victimDf)
            victimStats.append({'Level':level,'Victim Inbound':victimReceived}) 
        victimStats = pd.DataFrame(victimStats).sort_values('Level')
        # Save to pickle file for fast reloading
        victimStats.to_pickle(picklePath)
    else:
        if os.path.exists(picklePath):
            victimStats = pd.read_pickle(picklePath)
        else:
            raise Exception("Error no cached pickle file. Run the function with useCachedBytes=False to recalcuate the df, and create the pickle file '" + os.path.basename(picklePath) + "'" )
    return victimStats

In [ ]:
## Get Reflector bytes received and sent
def getReflectorStats(reflectorFilenames,Protocol,useCachedBytes=False,BytesOrPackets="bytes",Tool="Own"):
    print("Getting Reflector bytes for " + Protocol + " from pcap files...")
    picklePath = os.path.dirname(os.path.dirname(reflectorFilenames[0])) + '/ReflectorBytes_' + Protocol + '_' + BytesOrPackets + '_' + Tool + '.pkl'
    if not useCachedBytes:
        pbar = ProgressBar(widgets=widgets)
        reflectorStats = []
        for reflectorFile in pbar(reflectorFilenames):
            # Get attack level
            level = int(re.search(r'\d+',reflectorFile).group(0)) # Find first number in string = level
            reflectorDf = pcapToDf(reflectorFile,True,Tool)
            if Protocol == "TFTP":
                # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
                reflectorDfReceived = reflectorDf.loc[(reflectorDf['UDP Source Port'] == 50040)]
                reflectorDfSent = reflectorDf.loc[(reflectorDf['UDP Destination Port'] == 50040)]
            elif Protocol == "SNMP":
                reflectorDfReceived = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['Info'].str.contains("getBulkRequest"))]
                reflectorDfSent = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['Info'].str.contains("get-response"))]
            elif Protocol == "SSDP":
                reflectorDfReceived = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['Destination'] != ipaddress.ip_address('239.255.255.250')) & (reflectorDf["Info"].str.contains("M-SEARCH"))]
                reflectorDfSent = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['Destination'] != ipaddress.ip_address('239.255.255.250')) & (reflectorDf["UDP Destination Port"] == 50040)]
            if BytesOrPackets == "bytes":
                reflectorReceived = reflectorDfReceived["Length"].sum()
                reflectorSent = reflectorDfSent["Length"].sum()
            else:
                reflectorReceived = len(reflectorDfReceived)
                reflectorSent = len(reflectorDfSent)
            reflectorStats.append({'Level':level,'Reflector Inbound':reflectorReceived,'Reflector Outbound':reflectorSent})
        reflectorStats = pd.DataFrame(reflectorStats).sort_values('Level')
        # Save to pickle file for fast reloading
        reflectorStats.to_pickle(picklePath)
    else:
        if os.path.exists(picklePath):
            reflectorStats = pd.read_pickle(picklePath)
        else:
            raise Exception("Error no cached pickle file. Run the function with useCachedBytes=False to recalcuate the df, and create the pickle file '" + os.path.basename(picklePath) + "'" )
    return reflectorStats

In [ ]:
## Data manipulation for presentation
def getStatDf(attackerStats,victimStats,reflectorStats):
    StatDf = pd.DataFrame(columns=pd.MultiIndex.from_tuples([("Victim", "Inbound"), ("Reflector", "Inbound"), ("Reflector", "Outbound"), ("Attacker", "Outbound")]))
    StatDf["Level"] = victimStats['Level']
    StatDf[("Victim","Inbound")] = victimStats['Victim Inbound']
    StatDf[("Reflector","Inbound")] = reflectorStats['Reflector Inbound']
    StatDf[("Reflector","Outbound")] = reflectorStats['Reflector Outbound']
    StatDf[("Attacker","Outbound")] = attackerStats['Attacker Outbound']
    StatDf['Amplification factor'] = StatDf[("Victim","Inbound")] / StatDf[("Attacker","Outbound")]
    StatDf['Packets Per Second'] = attackerStats['Packets Per Second']
    column_to_move = StatDf.pop("Level")
    StatDf.insert(0, "Level", column_to_move)
    return StatDf

In [ ]:
## Plotting
def PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets):
    # Get single index dataframe
    if "Packets Per Second" in attackerStats.columns.to_list():
        attackerStats.pop("Packets Per Second")
    df1 = pd.merge(attackerStats,victimStats,on="Level")
    df2 = pd.merge(df1,reflectorStats,on='Level')
    # Remove 'level' string in level column
    df2["Level"] = df2['Level']
    # Get maximum value in dataframe rounded up to highest exponent (i.e. up to nearest 100M for example)
    maxN = df2.select_dtypes(include=[np.number]).max().max()
    c = 10 ** int(math.log10(maxN)) # Same number of digits as max number
    yaxisRange = [10,math.ceil(maxN/c) * c]

    # Plot data
    fig = px.line(df2, 
                title=Protocol + " Test traffic in " + BytesOrPackets,
                x='Level',
                y=['Attacker Outbound','Victim Inbound', 'Reflector Inbound','Reflector Outbound'],
                log_y=True,
                range_y=yaxisRange,
                markers=True,
                color_discrete_map={
                 "Attacker Outbound": "#ED2939",
                 "Victim Inbound": "#007FFF",
                 "Reflector Inbound":"#00A86B",
                 "Reflector Outbound":"orange" 
             })
    fig.update_layout(
        yaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text=BytesOrPackets.capitalize(),
            titlefont=dict(size=24,color="black")),
        xaxis=dict(
            tickfont=dict(size=16,color="black"),
            title_text="Level",
            titlefont=dict(size=24,color="black")),
        title={'font': {'size': 36,'color':"black"}},
        legend = dict(font = dict(size = 20, color = "black",)),
        legend_title = dict(text="Legend",font = dict(size = 24,color="black"))
        )
    fig.update_traces(line=dict(width=3),marker=dict(size=10))
    fig.show()
    fig.write_image(Protocol + " Test traffic in " + BytesOrPackets + ".pdf", width=1920/2.4, height=1080/2.4,scale=-2 ,engine='kaleido')

# Bytes

## TFTP DATA OWN TOOL

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_tftp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "TFTP"
useCachedBytes = True
BytesOrPackets="bytes"
Tool="own"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

## SNMP DATA OWN TOOL

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="own"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

## SSDP DATA OWN TOOL

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "bytes"
Tool="own"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

# Packets

## TFTP DATA OWN TOOL

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_tftp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "TFTP"
useCachedBytes = True
BytesOrPackets="packets"
Tool="own"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

## SNMP DATA OWN TOOL

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="own"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

## SSDP DATA OWN TOOL

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_own_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SSDP"
useCachedBytes = True
BytesOrPackets = "packets"
Tool="own"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

# LindenHoff Tool

## Bytes

### SNMP

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = False
BytesOrPackets = "bytes"
Tool="lindenhof"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

## SSDP

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SSDP"
useCachedBytes = False
BytesOrPackets = "bytes"
Tool="lindenhof"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

## Packets

### SNMP

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = False
BytesOrPackets = "packets"
Tool="lindenhof"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)

## SSDP

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SSDP"
useCachedBytes = False
BytesOrPackets = "packets"
Tool="lindenhof"

### SNMP

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_lindenhof_tool')
assert len(attackerFilenames) == len(victimFilenames) == len(reflectorFilenames)
Protocol = "SNMP"
useCachedBytes = False
BytesOrPackets = "bytes"
Tool="lindenhof"

### Reading the data

In [ ]:
# Read pcap 
attackerStats = getAttackStats(attackerFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
victimStats = getVictimStats(victimFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)
reflectorStats = getReflectorStats(reflectorFilenames,Protocol,useCachedBytes,BytesOrPackets,Tool)

### Displaying the data in latex

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerStats,victimStats,reflectorStats)
latexTableString = StatDf.to_latex(column_format='ccrrrcc',index=False,caption=Protocol + " ARDoS test traffic in " + BytesOrPackets + " for " + Tool + " tool",label=Protocol + "_Test_Traffic_in_" + BytesOrPackets + "_" + Tool,position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

### Plotting

In [ ]:
PlotLevelAttackData(attackerStats,victimStats,reflectorStats,Protocol,BytesOrPackets)